In [ ]:
 !pip install keras-tuner
!pip install csbdeep

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.4 MB/s 
     |████████████████████████████████| 2.9 MB 11.8 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff
import re
import zipfile
import scipy.io
import random
from tifffile import imread
import argparse
import json
import pickle as pkl
import imageio
import itertools
import jsonschema
import keras
import pathlib
from tensorflow.image import ssim as ssim2
from tensorflow.image import ssim_multiscale as msssim
from tqdm.utils import IS_WIN
from tqdm.keras import TqdmCallback as _TqdmCallback
import functools
import warnings
import tqdm
import keras.backend as K
import tensorflow as tf
import collections
from importlib import import_module
from tensorflow import __version__ as _tf_version
from tensorflow.python.client.device_lib import list_local_devices
from packaging import version
from keras.utils.conv_utils import normalize_tuple
import tensorflow_probability as tfp
import fractions
import numexpr
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import constant_op
import keras_tuner as kt


In [ ]:
# Open drive dynamically; prefer drives that have 'drive' in their name.
MAIN_DIR= r"/objective_transfer/deep_learning"
from google.colab import drive
drive.mount('/content/drive')
os.chdir(MAIN_DIR+'/CAREstd/')

Mounted at /content/drive


In [ ]:


def _verify_compatible_image_shapes(img1, img2):
    '''Checks if two image tensors are compatible for applying LWSSIM.
    This function checks if two sets of images have ranks at least 3, and if the
    last three dimensions match.
    Args:
      img1: Tensor containing the first image batch.
      img2: Tensor containing the second image batch.
    Returns:
      A tuple containing: the first tensor shape, the second tensor shape, and a
      list of control_flow_ops.Assert() ops implementing the checks.
    Raises:
      ValueError: When static shape check fails.
    '''
    shape1 = img1.get_shape().with_rank_at_least(3)
    shape2 = img2.get_shape().with_rank_at_least(3)
    shape1[-3:].assert_is_compatible_with(shape2[-3:])

    if shape1.ndims is not None and shape2.ndims is not None:
        for dim1, dim2 in zip(
            reversed(shape1.dims[:-3]), reversed(shape2.dims[:-3])):
          if not (dim1 == 1 or dim2 == 1 or dim1.is_compatible_with(dim2)):
              raise ValueError('Two images are not compatible: %s and %s' %
                              (shape1, shape2))

    # Now assign shape tensors.
    shape1, shape2 = array_ops.shape_n([img1, img2])

    # TODO(sjhwang): Check if shape1[:-3] and shape2[:-3] are broadcastable.
    checks = []
    checks.append(
        control_flow_ops.Assert(
            math_ops.greater_equal(array_ops.size(shape1), 3), [shape1, shape2],
            summarize=10))
    checks.append(
        control_flow_ops.Assert(
            math_ops.reduce_all(math_ops.equal(shape1[-3:], shape2[-3:])),
            [shape1, shape2],
            summarize=10))
    return shape1, shape2, checks

def mixconv(x, filters, strides=[1, 1, 1, 1], padding='SAME'):
    ''' x: input features with shape [N,H,W,C]
    filters: a list of filters with shape [K_i, K_i, C_i, M_i] for i−th group.
    Reference:
    https://arxiv.org/pdf/1907.09595.pdf
    '''
    # if tf.shape(x)[-1] != len(filters):
    #     raise ValueError('The last dimension of "x" must be equal to the length of the filters.')

    G = len(filters) # number of groups.
    y = []
    for xi, fi in zip(tf.split(x, G, axis=-1), filters):
        y.append(tf.nn.depthwise_conv2d(xi, fi, strides, padding))
    return tf.concat(y,axis=-1)
    # return y

### Helper functions from Tensorflow SSIM implementation

def _ssim_helper(x, y, reducer, max_val, compensation=1.0, k1=0.01, k2=0.03):
    '''Helper function for computing SSIM, copied from the Tensorflow ssim implementation.
    SSIM estimates covariances with weighted sums.  The default parameters
    use a biased estimate of the covariance:
    Suppose `reducer` is a weighted sum, then the mean estimators are
      \mu_x = \sum_i w_i x_i,
      \mu_y = \sum_i w_i y_i,
    where w_i's are the weighted-sum weights, and covariance estimator is
      cov_{xy} = \sum_i w_i (x_i - \mu_x) (y_i - \mu_y)
    with assumption \sum_i w_i = 1. This covariance estimator is biased, since
      E[cov_{xy}] = (1 - \sum_i w_i ^ 2) Cov(X, Y).
    For SSIM measure with unbiased covariance estimators, pass as `compensation`
    argument (1 - \sum_i w_i ^ 2).
    Args:
      x: First set of images, same size as y.
      y: Second set of images, same size as x.
      reducer: Reduces tensor of shape x Function that computes 'local' averages from the set of images. For
        non-convolutional version, this is usually tf.reduce_mean(x, [1, 2]), and
        for convolutional version, this is usually tf.nn.avg_pool2d or
        tf.nn.conv2d with weighted-sum kernel.
      max_val: The dynamic range (i.e., the difference between the maximum
        possible allowed value and the minimum allowed value).
      compensation: Compensation factor. See above.
      k1: Default value 0.01
      k2: Default value 0.03 (SSIM is less sensitivity to K2 for lower values, so
        it would be better if we took the values in the range of 0 < K2 < 0.4).
    Returns:
      A pair containing the luminance measure, and the contrast-structure measure.
    '''

    c1 = (k1 * max_val) ** 2
    c2 = (k2 * max_val) ** 2

    # SSIM luminance measure is
    # (2 * mu_x * mu_y + c1) / (mu_x ** 2 + mu_y ** 2 + c1).
    mean0 = reducer(x)
    mean1 = reducer(y)
    num0 = mean0 * mean1 * 2.0
    den0 = math_ops.square(mean0) + math_ops.square(mean1)
    luminance = (num0 + c1) / (den0 + c1)

    # SSIM contrast-structure measure is
    #   (2 * cov_{xy} + c2) / (cov_{xx} + cov_{yy} + c2).
    # Note that `reducer` is a weighted sum with weight w_k, \sum_i w_i = 1, then
    #   cov_{xy} = \sum_i w_i (x_i - \mu_x) (y_i - \mu_y)
    #          = \sum_i w_i x_i y_i - (\sum_i w_i x_i) (\sum_j w_j y_j).
    num1 = reducer(x * y) * 2.0
    den1 = reducer(math_ops.square(x) + math_ops.square(y))
    c2 *= compensation
    cs = (num1 - num0 + c2) / (den1 - den0 + c2)

    # SSIM score is the product of the luminance and contrast-structure measures.
    return luminance, cs

def _fspecial_gauss(size, sigma, cache = True):
    '''Function to mimic the 'fspecial' gaussian MATLAB function.'''

    # Cache results based on size and sigma. A nested function is used instead
    # of a global to not pollute the global namespace.
    # def get_cache(cache = None):
    #     # Mutated function parameters persist between calls.
    #     if cache == None:
    #         cache = {}
    #     if cache[(size, sigma)] == None:
    #         kernel = _fspecial_gauss(size, sigma, cache=False)
    #         cache[(size, sigma)] = kernel
    #     return cache[(size, sigma)]
    # if cache: return get_cache()

    size = tf.convert_to_tensor(size, tf.int32)
    sigma = tf.convert_to_tensor(sigma)

    coords = tf.cast(tf.range(size), sigma.dtype)

    coords -= tf.cast(size - 1, sigma.dtype) / 2.0

    g = math_ops.square(coords)
    g *= -0.5 / math_ops.square(sigma)

    g = tf.reshape(g, shape=[1, -1]) + tf.reshape(g, shape=[-1, 1])
    g = tf.reshape(g, shape=[1, -1])  # For tf.nn.softmax().
    g = tf.nn.softmax(g)
    return tf.reshape(g, shape=[size, size, 1, 1])

### Tensor-Operation Helpers

# def element_wise_weighted_sum(mat, weights):
#     '''Replaces each element with the sum of the weights multiplied by that element.
#     In other words: mat_i' = Σ mat_i x weights
#     Args:
#         mat: The matrix to be operated on.
#         weights: A non-empty, 1D vector that is multiplied by each element and summed to replace it.
#     Returns:
#         The modified matrix.
#     Raises:
#         ValueError if the weights' dimension isn't one.
#     '''

#     if np.ndim(weights) != 1:
#         raise ValueError('Weights must be 1D')

#     original_shape = mat.shape
#     # Flatten, tile and reshape into original shape with extended dimensions.
#     mat = tf.tile(tf.reshape(mat, [-1]), [len(weights)])
#     mat = tf.reshape(mat, [len(weights)] + original_shape)

#     # Multiply & Reduce
#     broadcast = [...] + [ None for _ in range(len(original_shape)) ]
#     mat = mat * weights[broadcast]
#     mat = tf.reduce_sum(mat, axis=[0])

#     assert mat.shape == original_shape, \
#            f'Result has unexpected shape: {mat.shape} ≠ {original_shape}'

#     return mat

#### LWSSIM

def _lwssim_per_channel(img1,
                        img2,
                        weights,
                        max_val=1.0,
                        filter_sizes=[11,11,11,11],
                        filter_sigma=1.5,
                        k1=0.01,
                        k2=0.03):
    '''Computes the level-weighted SSIM index between img1 and img2 per color channel.
    Details:
      - 11x11 Gaussian filter of width 1.5 is used.
      - k1 = 0.01, k2 = 0.03 as in the original paper.
    Args:
      img1: First image batch.
      img2: Second image batch.
      max_val: The dynamic range of the images (i.e., the difference between the
        maximum the and minimum allowed values).
      filter_sizes: Default value 11 (size of gaussian filter).
      filter_sigma: Default value 1.5 (width of gaussian filter).
      k1: Default value 0.01
      k2: Default value 0.03 (SSIM is less sensitivity to K2 for lower values, so
        it would be better if we took the values in the range of 0 < K2 < 0.4).
    Returns:
      A pair of tensors containing and channel-wise SSIM and contrast-structure
      values. The shape is [..., channels].
    '''
    filter_sizes = constant_op.constant(filter_sizes, dtype=dtypes.int32)
    filter_sigma = constant_op.constant(filter_sigma, dtype=img1.dtype)

    shape1, shape2 = array_ops.shape_n([img1, img2])
    checks = [
        control_flow_ops.Assert(
            math_ops.reduce_all(
                math_ops.greater_equal(shape1[-3:-1], tf.math.reduce_max(filter_sizes))),
            [shape1, tf.math.reduce_max(filter_sizes)],
            summarize=8),
        control_flow_ops.Assert(
            math_ops.reduce_all(
                math_ops.greater_equal(shape2[-3:-1], tf.math.reduce_max(filter_sizes))),
            [shape2, tf.math.reduce_max(filter_sizes)],
            summarize=8),
        # Ensure that there is just one channel.
        # control_flow_ops.Assert(
        #     shape1[-1] == 1,
        #     [shape1])
    ]

    # Enforce the check to run before computation.
    with ops.control_dependencies(checks):
        img1 = array_ops.identity(img1)


    def make_filter(filter_size):
        kernel = _fspecial_gauss(filter_size, filter_sigma)
        # TODO: Do in matrix operations ideally
        kernel = tf.tile(kernel, multiples=[1, 1, 1, 1])
        return kernel
    filters = []
    for i in range(len(filter_sizes)):
      filters.append(make_filter(filter_sizes[i]))

    # The correct compensation factor is `1.0 - tf.reduce_sum(tf.square(kernel))`,
    # but to match MATLAB implementation of MS-SSIM, we use 1.0 instead.
    compensation = 1.0

    def reducer(x):
        shape = tf.shape(x)
        # Reshape to get correct 4D tensor to conv2d.
        x = tf.reshape(x, shape=tf.concat([[-1], shape[-3:]], 0))
        #y = tf.nn.depthwise_conv2d(x, kernel, strides=[1, 1, 1, 1], padding='VALID')
        y = mixconv(x , filters)
        # Switch back to the original shape.
        return array_ops.reshape(
            y, array_ops.concat([shape[:-3], array_ops.shape(y)[1:]], 0))
        # return [array_ops.reshape(
        #     y[j], array_ops.concat([shape[:-3], array_ops.shape(y[j])[1:]], 0)) for j in range(len(filter_sizes))]

    extended_img1 = tf.repeat(img1, len(filter_sizes), axis=-1)
    extended_img2 = tf.repeat(img2, len(filter_sizes), axis=-1)

    luminance, cs = _ssim_helper(extended_img1, extended_img2, reducer, max_val, compensation, k1,
                                 k2)

    # Average over the second and the third from the last: height, width.
    axes = tf.constant([-3, -2], dtype=tf.int32)

    SSIM_temp = luminance * cs
    broadcast = [ None for _ in range(len(shape1)-1)] + [...]
    # Broadcasting so the weights multiply across channel which in this case are
    # the various sized filters
    Weighted_SSIM = SSIM_temp * weights[broadcast]
    Weighted_SSIM = tf.reduce_sum(Weighted_SSIM, axis=-1)
    ssim_val = math_ops.reduce_mean(Weighted_SSIM / len(weights), axes)
    cs = math_ops.reduce_mean(cs, axes)

    return ssim_val, cs

def get_weights(weights = None, seed = 0, weight_num = 4):
    # The weights won't be None if make_weights has already been called.
    if weights == None:
        tf.random.set_seed(seed)
        # Get uniform random weights.
        random_weights = tf.random.uniform(
            [weight_num],
            minval=0,
            maxval=1,
        )
        # Make them sum to 1.
        weights = random_weights / tf.reduce_sum(random_weights)

        # Log
        print(f'Computed LWSSIM weights: {weights}')

    return weights

def lwssim(img1,
           img2,
           max_val,
           weights=get_weights(),
           filter_sizes=[4,8,11,15],
           filter_sigma=1.5,
           k1=0.01,
           k2=0.03):
    # TODO: Change to specify that there must only be one channel.
    '''Computes the Level-Weighted SSIM index between img1 and img2.

    Note: The true SSIM is only defined on grayscale.  This function does not
    perform any colorspace transform.  (If the input is already YUV, then it will
    compute YUV SSIM average.)
    Details:
      - 11x11 Gaussian filter of width 1.5 is used.
      - k1 = 0.01, k2 = 0.03 as in the SSIM original paper.
    The image sizes must be at least 11x11 because of the filter size.
    Example:
    ```python
        # Read images (of size 255 x 255) from file.
        im1 = tf.image.decode_image(tf.io.read_file('path/to/im1.png'))
        im2 = tf.image.decode_image(tf.io.read_file('path/to/im2.png'))
        tf.shape(im1)  # `img1.png` has 3 channels; shape is `(255, 255, 3)`
        tf.shape(im2)  # `img2.png` has 3 channels; shape is `(255, 255, 3)`
        # Add an outer batch for each image.
        im1 = tf.expand_dims(im1, axis=0)
        im2 = tf.expand_dims(im2, axis=0)
        # Compute SSIM over tf.uint8 Tensors.
        ssim1 = lwssim(im1, im2, max_val=255, filter_sizes=[11],
                              filter_sigma=1.5, k1=0.01, k2=0.03)
        # Compute SSIM over tf.float32 Tensors.
        im1 = tf.image.convert_image_dtype(im1, tf.float32)
        im2 = tf.image.convert_image_dtype(im2, tf.float32)
        ssim2 = lwssim(im1, im2, max_val=1.0, filter_sizes=[11],
                              filter_sigma=1.5, k1=0.01, k2=0.03)
        # ssim1 and ssim2 both have type tf.float32 and are almost equal.
    ```
    Args:
      img1: First image batch. 4-D Tensor of shape `[batch, height, width,
        channels]` with only Positive Pixel Values.
      img2: Second image batch. 4-D Tensor of shape `[batch, height, width,
        channels]` with only Positive Pixel Values.
      max_val: The dynamic range of the images (i.e., the difference between the
        maximum the and minimum allowed values).
      weights: A non-empty, 1D vector that will modify
      filter_sizes: Default value 11 (size of gaussian filter).
      filter_sigma: Default value 1.5 (width of gaussian filter).
      k1: Default value 0.01
      k2: Default value 0.03 (SSIM is less sensitivity to K2 for lower values, so
        it would be better if we took the values in the range of 0 < K2 < 0.4).
    Returns:
      A tensor containing an SSIM value for each image in batch.  Returned SSIM
      values are in range (-1, 2], when pixel values are non-negative. Returns
      a tensor with shape: broadcast(img1.shape[:-3], img2.shape[:-3]).
    References:
        https://doi.org/10.48550/arXiv.1904.13362
    '''

    with ops.name_scope(None, 'LWSSIM', [img1, img2]):
        # Convert to tensor if needed.
        img1 = ops.convert_to_tensor(img1, name='img1')
        img2 = ops.convert_to_tensor(img2, name='img2')
        # Shape checking.
        _, _, checks = _verify_compatible_image_shapes(img1, img2)
        with ops.control_dependencies(checks):
          img1 = array_ops.identity(img1)

        # Need to convert the images to float32.  Scale max_val accordingly so that
        # SSIM is computed correctly.
        max_val = math_ops.cast(max_val, img1.dtype)
        max_val = tf.image.convert_image_dtype(max_val, dtypes.float32)
        img1 = tf.image.convert_image_dtype(img1, dtypes.float32)
        img2 = tf.image.convert_image_dtype(img2, dtypes.float32)

        ssim_per_channel, _ = _lwssim_per_channel(img1, img2, weights, max_val, filter_sizes,
                                                filter_sigma, k1, k2)

        value = math_ops.reduce_mean(ssim_per_channel, [-1])

        # Validate range.
        # if not np.isinf(value):
        #     assert -1 < value <= 2, \
        #           f'Computed LWSSIM value "{value}" outside of range (-1, 2].'

        return value

def lwssim_loss(y_true, y_pred):
    lwssim_val = lwssim(y_true, y_pred, 1, k2=0.05)
    normalized_lwssim = (lwssim_val + 1) / 2
    return 1-normalized_lwssim


def save_tiff(filename, image, format):
    {
        'imagej': save_imagej_hyperstack,
        'ome': save_ome_tiff
    }[format](filename, image)

def save_ome_tiff(filename, image):
    assert image.ndim in [3, 4]
    image = np.expand_dims(image, (1, 2) if image.ndim == 3 else 1)
    c, t, z, y, x = image.shape

    pixel_type = {
        np.dtype('uint8'): 'Uint8',
        np.dtype('uint16'): 'Uint16',
        np.dtype('float32'): 'Float'
    }[image.dtype]

    channel_names = ['Raw', 'Restored', 'Ground Truth']
    lsid_base = 'ome.drvtechnologies.com:'

    channel_info = ''
    for i, name in enumerate(channel_names[:c]):
        channel_info += f'''\
    <ChannelInfo Name="{name}" ID="{lsid_base}ChannelInfo:{i + 3}">
      <ChannelComponent Index="{i}" Pixels="{lsid_base}Pixels:2"/>
    </ChannelInfo>
'''
    description = f'''\
<OME xmlns="http://www.openmicroscopy.org/XMLschemas/OME/FC/ome.xsd">
  <Image Name="Unnamed [{pixel_type} {x}x{y}x{z}x{t} Channels]"
         ID="{lsid_base}Image:1">
{channel_info}\
    <Pixels DimensionOrder="XYZTC" PixelType="{pixel_type}"
            SizeX="{x}" SizeY="{y}" SizeZ="{z}" SizeT="{t}" SizeC="{c}"
            BigEndian="false" ID="{lsid_base}Pixels:2">
      <TiffData IFD="0" NumPlanes="{z * c * t}"/>
    </Pixels>
  </Image>
</OME>
'''

    tiff.imwrite(
        filename,
        data=image,
        description=description,
        metadata=None)

def save_imagej_hyperstack(filename, image):
    assert image.ndim in [3, 4]
    if image.ndim == 4:
        image = np.transpose(image, (3, 0, 1, 2))

    tiff.imwrite(str(filename)+'.tif', image, imagej=True)

def _get_spatial_ndim(x):
    return keras.backend.ndim(x) - 2

def _get_num_channels(x):
    return keras.backend.int_shape(x)[-1]

def mae(y_true, y_pred):
    '''Mean absolute error'''
    return keras.losses.mae(
        *[keras.backend.batch_flatten(y) for y in [y_true, y_pred]])


def mse(y_true, y_pred):
    '''Mean squared error'''
    return keras.losses.mse(
        *[keras.backend.batch_flatten(y) for y in [y_true, y_pred]])

def _conv(x, num_filters, kernel_size, padding='same', **kwargs):
    n = _get_spatial_ndim(x)
    if n not in (2, 3):
        raise NotImplementedError(f'{n}D convolution is not supported')

    return (keras.layers.Conv2D if n == 2 else
            keras.layers.Conv3D)(
        num_filters, kernel_size, padding=padding, **kwargs)(x)

def _global_average_pooling(x):
    n = _get_spatial_ndim(x)
    if n == 2:
        return keras.layers.GlobalAveragePooling2D()(x)
    elif n == 3:
        return keras.layers.GlobalAveragePooling3D()(x)
    else:
        raise NotImplementedError(
            f'{n}D global average pooling is not supported')


def _channel_attention_block(x, reduction):
    '''
    Channel attention block.
    References
    ----------
    - Squeeze-and-Excitation Networks
      https://arxiv.org/abs/1709.01507
    - Image Super-Resolution Using Very Deep Residual Channel Attention
      Networks
      https://arxiv.org/abs/1807.02758
    '''

    num_channels = _get_num_channels(x)

    y = _global_average_pooling(x)
    y = keras.layers.Reshape((*(1,) * _get_spatial_ndim(x), num_channels))(y)
    y = _conv(y, num_channels // reduction, 1, activation='relu')
    y = _conv(y, num_channels, 1, activation='sigmoid')

    return keras.layers.Multiply()([x, y])

# _tf_version = version.parse(_tf_version)
# IS_TF_1 = _tf_version.major == 1
# _KERAS = 'keras' if IS_TF_1 else 'tensorflow.keras'

def _residual_channel_attention_blocks(x,
                                       repeat=1,
                                       channel_reduction=8,
                                       residual_scaling=1.0):
    num_channels = _get_num_channels(x)

    for _ in range(repeat):
        skip = x

        x = _conv(x, num_channels, 3, activation='relu')
        x = _conv(x, num_channels, 3)

        x = _channel_attention_block(x, channel_reduction)

        if residual_scaling != 1.0:
            x = keras.layers.Lambda(lambda x: residual_scaling * x)(x)

        x = keras.layers.Add()([x, skip])

    return x
def normalize(image, p_min=2, p_max=99.9, dtype='float32'):
    '''
    Normalizes the image intensity so that the `p_min`-th and the `p_max`-th
    percentiles are converted to 0 and 1 respectively.
    References
    ----------
    Content-Aware Image Restoration: Pushing the Limits of Fluorescence
    Microscopy
    https://doi.org/10.1038/s41592-018-0216-7
    '''
    low, high = np.percentile(image, (p_min, p_max))
    return numexpr.evaluate(
        '(image - low) / (high - low + 1e-6)').astype(dtype)

def _normalize(x):
    '''
    Normalize the signal so that the range becomes [0, 1].
    '''
    prefix = 'lambda_Normalize'
    name = prefix + '_' + str(keras.backend.get_uid(prefix))
    return keras.layers.Lambda(lambda x: (x-tf.reduce_min(x, keepdims=True))/(tf.reduce_max(x, keepdims=True)-tf.reduce_min(x, keepdims=True)), name=name)(x)


def _standardize(x):
    '''
    Standardize the signal so that the range becomes [-1, 1] (assuming the
    original range is [0, 1]).
    '''
    prefix = 'lambda_standardize'
    name = prefix + '_' + str(keras.backend.get_uid(prefix))
    return keras.layers.Lambda(lambda x: 2 * x - 1, name=name)(x)


def _destandardize(x):
    '''Undo standardization'''
    prefix = 'lambda_destandardize'
    name = prefix + '_' + str(keras.backend.get_uid(prefix))
    return keras.layers.Lambda(lambda x: 0.5 * x + 0.5, name=name)(x)

def keras_import(sub=None, *names):
    if sub is None:
        return import_module(_KERAS)
    else:
        mod = import_module('{_KERAS}.{sub}'.format(_KERAS=_KERAS,sub=sub))
        if len(names) == 0:
            return mod
        elif len(names) == 1:
            return getattr(mod, names[0])
        return tuple(getattr(mod, name) for name in names)

def build_rcan(input_shape=(16, 256, 256, 1),
               *,
               num_channels=32,
               num_residual_blocks=3,
               num_residual_groups=5,
               channel_reduction=8,
               residual_scaling=1.0,
               num_output_channels=-1):
    '''
    Builds a residual channel attention network. Note that the upscale module
    at the end of the network is omitted so that the input and output of the
    model have the same size.
    Parameters
    ----------
    input_shape: tuple of int
        Input shape of the model.
    num_channels: int
        Number of feature channels.
    num_residual_blocks: int
        Number of residual channel attention blocks in each residual group.
    num_residual_groups: int
        Number of residual groups.
    channel_reduction: int
        Channel reduction ratio for channel attention.
    residual_scaling: float
        Scaling factor applied to the residual component in the residual
        channel attention block.
    num_output_channels: int
        Number of channels in the output image. if negative, it is set to the
        same number as the input.
    Returns
    -------
    keras.Model
        Keras model instance.
    References
    ----------
    Image Super-Resolution Using Very Deep Residual Channel Attention Networks
    https://arxiv.org/abs/1807.02758
    '''

    if num_output_channels < 0:
        num_output_channels = input_shape[-1]

    inputs = keras.layers.Input(input_shape)
    #x = _normalize(inputs)
    x = _standardize(inputs)
    x = _conv(x, num_channels, 3)

    long_skip = x

    for _ in range(num_residual_groups):
        short_skip = x

        x = _residual_channel_attention_blocks(
            x,
            num_residual_blocks,
            channel_reduction,
            residual_scaling)

        if num_residual_groups == 1:
            break

        x = _conv(x, num_channels, 3)
        x = keras.layers.Add()([x, short_skip])

    x = _conv(x, num_channels, 3)
    x = keras.layers.Add()([x, long_skip])

    x = _conv(x, num_output_channels, 3)
    outputs = _destandardize(x)

    return keras.Model(inputs, outputs)

def SSIM_loss(y_true, y_pred):
    return 1-((ssim2(y_true, y_pred,1,k2=0.05)+1)*0.5) # change to 2

def SSIML1_loss(y_true, y_pred, alpha=0.84):
    SSIM = 1-((ssim2(y_true, y_pred,1,k2=0.05)+1)*0.5)
    MAE = keras.losses.mae(
        *[keras.backend.batch_flatten(y) for y in [y_true, y_pred]])
    result =  alpha * SSIM + (1-alpha) * MAE

    return result


def axes_check_and_normalize(axes,length=None,disallowed=None,return_allowed=False):
    """
    S(ample), T(ime), C(hannel), Z, Y, X
    """
    allowed = 'STCZYX'
    axes is not None or _raise(ValueError('axis cannot be None.'))
    axes = str(axes).upper()
    consume(a in allowed or _raise(ValueError("invalid axis '%s', must be one of %s."%(a,list(allowed)))) for a in axes)
    disallowed is None or consume(a not in disallowed or _raise(ValueError("disallowed axis '%s'."%a)) for a in axes)
    consume(axes.count(a)==1 or _raise(ValueError("axis '%s' occurs more than once."%a)) for a in axes)
    length is None or len(axes)==length or _raise(ValueError('axes (%s) must be of length %d.' % (axes,length)))
    return (axes,allowed) if return_allowed else axes

def consume(iterator):
    collections.deque(iterator, maxlen=0)

def _raise(e):
    if isinstance(e, BaseException):
        raise e
    else:
        raise ValueError(e)

def axes_dict(axes):
    """
    from axes string to dict
    """
    axes, allowed = axes_check_and_normalize(axes,return_allowed=True)
    return { a: None if axes.find(a) == -1 else axes.find(a) for a in allowed }
    # return collections.namedtuple('Axes',list(allowed))(*[None if axes.find(a) == -1 else axes.find(a) for a in allowed ])

def backend_channels_last():
    K = keras_import('backend')
    assert K.image_data_format() in ('channels_first','channels_last')
    return K.image_data_format() == 'channels_last'

def move_channel_for_backend(X,channel):
    if backend_channels_last():
        return np.moveaxis(X, channel, -1)
    else:
        return np.moveaxis(X, channel,  1)

def get_gpu_count():
    '''Returns the number of available GPUs.'''
    return len([x for x in list_local_devices() if x.device_type == 'GPU'])


def is_multi_gpu_model(model):
    '''Checks if the model supports multi-GPU data parallelism.'''
    return hasattr(model, 'is_multi_gpu_model') and model.is_multi_gpu_model


def convert_to_multi_gpu_model(model, gpus=None):
    '''
    Converts a model into a multi-GPU version if possible.
    Parameters
    ----------
    model: keras.Model
        Model to be converted.
    gpus: int or None
        Number of GPUs used to create model replicas. If None, all GPUs
        available on the device will be used.
    Returns
    -------
    keras.Model
        Multi-GPU model.
    '''

    gpus = gpus or get_gpu_count()

    if gpus <= 1 or is_multi_gpu_model(model):
        return model

    multi_gpu_model = keras.utils.multi_gpu_model(
        model, gpus=gpus, cpu_relocation=True)

    # copy weights
    multi_gpu_model.layers[
        -(len(multi_gpu_model.outputs) + 1)].set_weights(model.get_weights())

    setattr(multi_gpu_model, 'is_multi_gpu_model', True)
    setattr(multi_gpu_model, 'gpus', gpus)

    return multi_gpu_model

def psnr(y_true, y_pred):
    '''
    Computs the peak signal-to-noise ratio between two images. Note that the
    maximum signal value is assumed to be 1.
    '''
    # p, q = [K.batch_flatten(y) for y in [y_true, y_pred]]
    psnr2 = tf.image.psnr(y_true, y_pred, max_val=1.0)
    return psnr2
    # return -4.342944819 * K.log(K.mean(K.square(p - q), axis=-1))


def ssim(y_true, y_pred):
    '''
    Computes the structural similarity index between two images. Note that the
    maximum signal value is assumed to be 1.
    References
    ----------
    Image Quality Assessment: From Error Visibility to Structural Similarity
    https://doi.org/10.1109/TIP.2003.819861
    '''

    # c1 = 0.01 ** 2
    # c2 = 0.03 ** 2

    # dim = K.ndim(y_pred) - 2
    # if dim not in (2, 3):
    #     raise NotImplementedError(f'{dim}D SSIM is not suported')

    # num_channels = K.int_shape(y_pred)[-1]

    # kernel = _get_gaussian_kernel(dim, 11, 1.5)
    # conv = K.conv2d if dim == 2 else K.conv3d

    # def average(x):
    #     # channel-wise weighted average using the Gaussian kernel
    #     return tf.concat(
    #         [conv(y, kernel) for y in tf.split(x, num_channels, axis=-1)],
    #         axis=-1)

    # ux = average(y_true)
    # uy = average(y_pred)

    # a = ux * uy
    # b = K.square(ux) + K.square(uy)
    # c = average(y_true * y_pred)
    # d = average(K.square(y_true) + K.square(y_pred))

    # lum = (2 * a + c1) / (b + c1)
    # cs = (2 * (c - a) + c2) / (d - b + c2)

    # return K.mean(K.batch_flatten(lum * cs), axis=-1)
    return ssim2(y_true, y_pred,1,k2=0.05)

def staircase_exponential_decay(n):
    '''
    Returns a scheduler function to drop the learning rate by half
    every `n` epochs.
    '''
    return lambda epoch, lr: lr / 2 if epoch != 0 and epoch % n == 0 else lr

def load_training_data(file, validation_split=0, axes=None, n_images=None, verbose=False):
    """Load training data from file in ``.npz`` format.
    The data file is expected to have the keys:
    - ``X``    : Array of training input images.
    - ``Y``    : Array of corresponding target images.
    - ``axes`` : Axes of the training images.
    Parameters
    ----------
    file : str
        File name
    validation_split : float
        Fraction of images to use as validation set during training.
    axes: str, optional
        Must be provided in case the loaded data does not contain ``axes`` information.
    n_images : int, optional
        Can be used to limit the number of images loaded from data.
    verbose : bool, optional
        Can be used to display information about the loaded images.
    Returns
    -------
    tuple( tuple(:class:`numpy.ndarray`, :class:`numpy.ndarray`), tuple(:class:`numpy.ndarray`, :class:`numpy.ndarray`), str )
        Returns two tuples (`X_train`, `Y_train`), (`X_val`, `Y_val`) of training and validation sets
        and the axes of the input images.
        The tuple of validation data will be ``None`` if ``validation_split = 0``.
    """

    f = np.load(file)
    X, Y = f['X'], f['Y']
    if axes is None:
        axes = f['axes']
    axes = axes_check_and_normalize(axes)

    # assert X.shape == Y.shape
    assert X.ndim == Y.ndim
    assert len(axes) == X.ndim
    #assert 'C' in axes
    if n_images is None:
        n_images = X.shape[0]
    assert X.shape[0] == Y.shape[0]
    assert 0 < n_images <= X.shape[0]
    assert 0 <= validation_split < 1

    X, Y = X[:n_images], Y[:n_images]
    #channel = axes_dict(axes)['C']

    if validation_split > 0:
        #n_val   = int(round(n_images * validation_split))
        ''' NEED TO SET VALIDATION SPLIT BETTER '''
        n_val = 620
        n_train = n_images - n_val
        assert 0 < n_val and 0 < n_train
        X_t, Y_t = X[-n_val:],  Y[-n_val:]
        X,   Y   = X[:n_train], Y[:n_train]
        assert X.shape[0] == n_train and X_t.shape[0] == n_val
        #X_t = move_channel_for_backend(X_t,channel=channel)
        #Y_t = move_channel_for_backend(Y_t,channel=channel)

    #X = move_channel_for_backend(X,channel=channel)
    #Y = move_channel_for_backend(Y,channel=channel)

    # axes = axes.replace('C','') # remove channel
    # if backend_channels_last():
    #     axes = axes+'C'
    # else:
    #     axes = axes[:1]+'C'+axes[1:]

    data_val = (X_t,Y_t) if validation_split > 0 else None

    if verbose:
        ax = axes_dict(axes)
        n_train, n_val = len(X), len(X_t) if validation_split>0 else 0
        image_size = tuple( X.shape[ax[a]] for a in axes if a in 'TZYX' )
        n_dim = len(image_size)
        #n_channel_in, n_channel_out = X.shape[ax['C']], Y.shape[ax['C']]

        print('number of training images:\t', n_train)
        print('number of validation images:\t', n_val)
        print('image size (%dD):\t\t'%n_dim, image_size)
        print('axes:\t\t\t\t', axes)
        #print('channels in / out:\t\t', n_channel_in, '/', n_channel_out)

    return (X,Y), data_val, axes

class ModelCheckpoint(tf.keras.callbacks.ModelCheckpoint):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save >= self.period:
            self.epochs_since_last_save = 0
            filepath = self.filepath.format(epoch=epoch + 1, **logs)

            if self.save_best_only:
                current = logs.get(self.monitor)
                if current is None:
                    warnings.warn('Can save best model only with %s '
                                  'available, skipping.'
                                  % (self.monitor), RuntimeWarning)
                else:
                    if self.monitor_op(current, self.best):
                        if self.verbose > 0:
                            print('\nEpoch %05d: %s improved from %0.5f to '
                                  '%0.5f, saving model to %s'
                                  % (epoch + 1, self.monitor, self.best,
                                     current, filepath))
                        self.best = current
                        save_model(
                            filepath, self.model, self.save_weights_only)
                    else:
                        if self.verbose > 0:
                            print('\nEpoch %05d: %s did not improve from %0.5f'
                                  % (epoch + 1, self.monitor, self.best))
            else:
                if self.verbose > 0:
                    print('\nEpoch %05d: saving model to %s'
                          % (epoch + 1, filepath))
                save_model(filepath, self.model, self.save_weights_only)


class TqdmCallback(_TqdmCallback):
    def __init__(self):
        super().__init__(
            tqdm_class=functools.partial(
                tqdm.tqdm, dynamic_ncols=True, ascii=IS_WIN))
        self.on_batch_end = self.bar2callback(
            self.batch_bar, pop=['batch', 'size'])

def save_model(filename, model, weights_only=False):
    if is_multi_gpu_model(model):
        m = model.layers[-(len(model.outputs) + 1)]
    else:
        m = model

    if weights_only:
        m.save_weights(filename, overwrite=True)
    else:
        m.save(filename, overwrite=True)

class DataGenerator:
    '''
    Generates batches of image pairs with real-time data augmentation.
    Parameters
    ----------
    shape: tuple of int
        Shape of batch images (excluding the channel dimension).
    batch_size: int
        Batch size.
    transform_function: str or callable or None
        Function used for data augmentation. Typically you will set
        ``transform_function='rotate_and_flip'`` to apply combination of
        randomly selected image rotation and flipping.  Alternatively, you can
        specify an arbitrary transformation function which takes two input
        images (source and target) and returns transformed images. If
        ``transform_function=None``, no augmentation will be performed.
    intensity_threshold: float
        If ``intensity_threshold > 0``, pixels whose intensities are greater
        than this threshold will be considered as foreground.
    area_ratio_threshold: float between 0 and 1
        If ``intensity_threshold > 0``, the generator calculates the ratio of
        foreground pixels in a target patch, and rejects the patch if the ratio
        is smaller than this threshold.
    scale_factor: int != 0
        Scale factor for the target patch size. Positive and negative values
        mean up- and down-scaling respectively.
    '''
    def __init__(self,
                 shape,
                 batch_size,
                 transform_function='rotate_and_flip',
                 intensity_threshold=0.0,
                 area_ratio_threshold=0.0,
                 scale_factor=1):
        def rotate_and_flip(x, y, dim):
            if dim == 2:
                k = np.random.randint(0, 4)
                x, y = [np.rot90(v, k=k) for v in (x, y)]
                if np.random.random() < 0.5:
                    x, y = [np.fliplr(v) for v in (x, y)]
                return x, y
            elif dim == 3:
                k = np.random.randint(0, 4)
                x, y = [np.rot90(v, k=k, axes=(1, 2)) for v in (x, y)]
                if np.random.random() < 0.5:
                    x, y = [np.flip(v, axis=1) for v in (x, y)]
                if np.random.random() < 0.5:
                    x, y = [np.flip(v, axis=0) for v in (x, y)]
                return x, y
            else:
                raise ValueError('Unsupported dimension')

        self._shape = tuple(shape)
        self._batch_size = batch_size

        dim = len(self._shape)

        if transform_function == 'rotate_and_flip':
            if shape[-2] != shape[-1]:
                raise ValueError(
                    'Patch shape must be square when using `rotate_and_flip`; '
                    f'Received shape: {shape}')
            self._transform_function = lambda x, y: rotate_and_flip(x, y, dim)
        elif callable(transform_function):
            self._transform_function = transform_function
        elif transform_function is None:
            self._transform_function = lambda x, y: (x, y)
        else:
            raise ValueError('Invalid transform function')

        self._intensity_threshold = intensity_threshold

        if not 0 <= area_ratio_threshold <= 1:
            raise ValueError('"area_ratio_threshold" must be between 0 and 1')
        self._area_threshold = area_ratio_threshold * np.prod(shape)

        self._scale_factor = normalize_tuple(scale_factor, dim, 'scale_factor')
        if any(not isinstance(f, int) or f == 0 for f in self._scale_factor):
            raise ValueError('"scale_factor" must be nonzero integer')

    class _Sequence(tf.keras.utils.Sequence):
        def _scale(self, shape):
            return tuple(
                s * f if f > 0 else s // -f
                for s, f in zip(shape, self._scale_factor))

        def __init__(self,
                     x,
                     y,
                     batch_size,
                     shape,
                     transform_function,
                     intensity_threshold,
                     area_threshold,
                     scale_factor):
            self._batch_size = batch_size
            self._transform_function = transform_function
            self._intensity_threshold = intensity_threshold
            self._area_threshold = area_threshold
            self._scale_factor = scale_factor

            for s, f, in zip(shape, self._scale_factor):
                if f < 0 and s % -f != 0:
                    raise ValueError(
                        'When downsampling, all elements in `shape` must be '
                        'divisible by the scale factor; '
                        f'Received shape: {shape}, '
                        f'scale factor: {self._scale_factor}')

            self._x, self._y = [
                list(m) if isinstance(m, (list, tuple)) else [m]
                for m in [x, y]]
            self._x = np.moveaxis(self._x,0,-1)
            self._y = np.moveaxis(self._y,0,-1)
            if len(self._x) != len(self._y):
                raise ValueError(
                    'Different number of images are given: '
                    f'{len(self._x)} vs. {len(self._y)}')

            if len({m.dtype for m in self._x}) != 1:
                raise ValueError('All source images must be the same type')
            if len({m.dtype for m in self._y}) != 1:
                raise ValueError('All target images must be the same type')
            print(len(self._x))
            for i in range(len(self._x)):
                if len(self._x[i].shape) == len(shape):
                    self._x[i] = self._x[i][..., np.newaxis]

                if len(self._y[i].shape) == len(shape):
                    self._y[i] = self._y[i][..., np.newaxis]

                if len(self._x[i].shape) != len(shape) + 1:
                    raise ValueError(f'Source image must be {len(shape)}D')

                if len(self._y[i].shape) != len(shape) + 1:
                    raise ValueError(f'Target image must be {len(shape)}D')
                if self._x[i].shape[:-1] < shape:
                    raise ValueError(
                        'Source image must be larger than the patch size')

                expected_y_image_size = self._scale(self._x[i].shape[:-1])
                if self._y[i].shape[:-1] != expected_y_image_size:
                    raise ValueError('Invalid target image size: '
                                     f'expected {expected_y_image_size}, '
                                     f'but received {self._y[i].shape[:-1]}')

            if len({m.shape[-1] for m in self._x}) != 1:
                raise ValueError(
                    'All source images must have the same number of channels')
            if len({m.shape[-1] for m in self._y}) != 1:
                raise ValueError(
                    'All target images must have the same number of channels')
            print()
            self._batch_x = np.zeros(
                (batch_size, *shape, self._x[0].shape[-1]),
                dtype=self._x[0].dtype)
            self._batch_y = np.zeros(
                (batch_size, *self._scale(shape),self._y[0].shape[-1]),
                dtype=self._y[0].dtype)

        def __len__(self):
            return len(self._x) // self._batch_size # return a dummy value

        def __next__(self):
            return self.__getitem__(0)

        def __getitem__(self, _):
            for i in range(self._batch_size):
                for _ in range(139):
                    j = np.random.randint(0, len(self._x))

                    tl = [np.random.randint(0, a - b + 1)
                          for a, b in zip(
                              self._x[j].shape, self._batch_x.shape[1:])]

                    x = np.copy(self._x[j][tuple(
                        [slice(a, a + b) for a, b in zip(
                            tl, self._batch_x.shape[1:])])])
                    y = np.copy(self._y[j][tuple(
                        [slice(a, a + b) for a, b in zip(
                            self._scale(tl), self._batch_y.shape[1:])])])

                    if (self._intensity_threshold <= 0.0 or
                            np.count_nonzero(y > self._intensity_threshold)
                            >= self._area_threshold):
                        break
                else:
                    warnings.warn(
                        'Failed to sample a valid patch',
                        RuntimeWarning,
                        stacklevel=3)

                self._batch_x[i], self._batch_y[i] = \
                    self._transform_function(x, y)
            return self._batch_x, self._batch_y

    def flow(self, x, y):
        '''
        Returns a `keras.utils.Sequence` object which generates batches
        infinitely. It can be used as an input generator for
        `keras.models.Model.fit_generator()`.
        Parameters
        ----------
        x: array_like or list of array_like
            Source image(s).
        y: array_like or list of array_like
            Target image(s).
        Returns
        -------
        keras.utils.Sequence
            `keras.utils.Sequence` object which generates tuples of source and
            target image patches.
        '''
        return self._Sequence(x,
                              y,
                              self._batch_size,
                              self._shape,
                              self._transform_function,
                              self._intensity_threshold,
                              self._area_threshold,
                              self._scale_factor)

def _get_gaussian_kernel(dim, size, sigma):
    k = size // 2
    normal = tfp.distributions.Normal(0.0, sigma)
    p = normal.prob(tf.range(-k, size - k, dtype=tf.float32))

    indices = [chr(i) for i in range(105, 105 + dim)]
    eq = ','.join(indices) + '->' + ''.join(indices)
    kernel = tf.einsum(eq, *([p] * dim))
    kernel /= tf.reduce_sum(kernel)
    kernel = kernel[..., tf.newaxis, tf.newaxis]

    return kernel


def load_data_care(path):
    # Loading preprocessed image patches and adding 4th arbitrary dimension
    b = np.load(MAIN_DIR+path)
    training_data = b['t']
    val_data = b['v']
    # test_data = b['te']
    if len(np.shape(training_data))>3:
      # for 3D only, changing dimensions
      training_data = np.transpose(training_data, (0, 3, 1, 2,4))
      val_data = np.transpose(val_data, (0, 3, 1, 2,4))
      # test_data = np.transpose(test_data, (0, 3, 1, 2,4))
    # test_data = np.expand_dims(test_data, -1)
    # res = [training_data, val_data, test_data]
    res = [training_data, val_data]
    return res

Computed LWSSIM weights: [0.18303473 0.12949334 0.33562824 0.35184368]


In [ ]:
class TuneModel(kt.HyperModel):
    def build(self, hp):
      hyperparameter_config = {
          'initial_learning_rate': hp.Float('initial_learning_rate', 1e-6, 1e-3, sampling='log'),
          'batch_size': hp.Choice("batch_size", [8,16,32]), # TODO: 70 may not work.
          'alpha': hp.Float("alpha", 0.5, 1, sampling='linear'), # TODO: Check if this is good enough
      }

      print(f'Training model with hyperparameters: {hyperparameter_config}')


      global config


      merged_config = { **config, **hyperparameter_config }
      self.merged_config = merged_config

      raw_loss = {'mae' : mae,'mse': mse,'SSIM_loss': SSIM_loss,'SSIML1_loss': SSIML1_loss}[merged_config['loss']]
      if merged_config['loss'] == 'SSIML1_loss':
          custom_objects = {"SSIM_loss": SSIM_loss}
          with tf.keras.utils.custom_object_scope(custom_objects):
              loss = lambda x, y: raw_loss(x, y, merged_config['alpha'])
      else:
          loss = raw_loss

      model.compile(
          optimizer=tf.keras.optimizers.Adam(lr=merged_config['initial_learning_rate']),
          loss=loss,
          metrics=[{'psnr': psnr, 'ssim': ssim}[m] for m in merged_config['metrics']])
      return model


    def fit(self, hp, model, *args, **kwargs):
        global model_save_path

        data_gen = DataGenerator(
            self.merged_config['input_shape'],
            self.merged_config['batch_size'],
            transform_function=None)

        training_data = data_gen.flow(*list(zip([X,Y])))
        validation_data = data_gen.flow(*list(zip([X_val,Y_val])))

        if validation_data is not None:
            checkpoint_filepath = 'weights_{epoch:03d}_{val_loss:.8f}.hdf5'
        else:
            checkpoint_filepath = 'weights_{epoch:03d}_{loss:.8f}.hdf5'



        output_dir = model_save_path
        print('Training RCAN model')
        return model.fit(
            x=training_data,
            epochs=epochs,
            #steps_per_epoch=steps_per_epoch,
            validation_data=validation_data,
            #validation_steps=validation_steps,
            verbose=1,
            callbacks=callbacks
            )

## **Execution**

In [ ]:
#################################################################################
''' Data was Normalized in Local Prep'''

model_name = 'NADH_model_0725_cervix_SSIMMAE_kt_hyper_fix'
main_path = MAIN_DIR

if not os.path.exists(os.path.join(main_path,model_name)): os.mkdir(os.path.join(main_path,model_name))
model_save_path = os.path.join(main_path,model_name)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
print(is_cuda_gpu_available)
(X,X_val) = load_data_care('/npz/Cervix_all_data_original.npz')
(Y,Y_val) = load_data_care('/npz/Cervix_all_data_labels_original.npz')
print(np.shape(X))
axes = 'SXY'
#c = axes_dict(axes)['C']
#n_channel_in, n_channel_out = X.shape[c], Y.shape[c]

config = {
    "epochs": 300,
    "steps_per_epoch": 4,
    "num_residual_groups": 5,
    "num_residual_blocks": 5,
    "input_shape": [256, 256],
    "training_data_images": {"raw":X,
                          "gt":Y},
    "validation_data_images": {"raw":X_val,
                            "gt":Y_val},
    "LW_weight_seed": 0,
    "LW_weight_num": 4
}


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Num GPUs Available:  1
True
(238, 512, 512)


## **Tuning**

In [ ]:
if False:

  # Keras tuner error of metric recovery (squencing error)
  config.setdefault('epochs', 300)
  config.setdefault('steps_per_epoch', 256)
  config.setdefault('initial_learning_rate', 1e-5)
  config.setdefault('loss','SSIML1_loss')
  config.setdefault('metrics', ['psnr','ssim'])
  config.setdefault('num_residual_blocks', 3)
  config.setdefault('num_residual_groups', 5)
  config.setdefault('channel_reduction', 4)
  config.setdefault('num_channels', 32)
  config.setdefault('LW_weight_seed', 0)
  config.setdefault('LW_weight_num', 4)


  input_shape = config['input_shape']
  print('Building RCAN model')
  print('  - input_shape =', input_shape)
  for s in ['num_channels',
            'num_residual_blocks',
            'num_residual_groups',
            'channel_reduction']:
      print(f'  - {s} =', config[s])

  strategy = tf.distribute.MirroredStrategy()
  print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

  with strategy.scope():
      model = build_rcan(
          (*input_shape,1),
          num_channels=config['num_channels'],
          num_residual_blocks=config['num_residual_blocks'],
          num_residual_groups=config['num_residual_groups'],
          channel_reduction=config['channel_reduction'])

      gpus = get_gpu_count()

      #model = convert_to_multi_gpu_model(model, gpus)
  output_dir = model_save_path

  checkpoint_filepath = 'Best_Hyperparameter_Model.hdf5'

  stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_ssim", patience=20) # TODO: Check if patience is right
  callbacks=[stop_early
          ]
  tuner = kt.BayesianOptimization(TuneModel(),
                          objective=kt.Objective("val_ssim", direction="max"),
                          max_trials=10, # TODO: Check if 10 is right
                          directory=model_save_path,
                          project_name="KT_Hyper")
  epochs = config['epochs']

  tuner.search(X=X,Y=Y,X_val=X_val,Y_val=Y_val, epochs=epochs, callbacks=callbacks)

  # Report results
  print('The hyperparameter search is complete; printing the best parameters in order.')
  for index, hyperparameters in enumerate(best_hps=tuner.get_best_hyperparameters()):
      print(f'Best {index+1} hyperparameters are: {hyperparameters.values}')

## **Training**

In [ ]:

#RCAN TRAINING
if True:
  config_rcan = {
      "epochs": 50,
      "steps_per_epoch": 4,
      "num_residual_groups": 5,
      "num_residual_blocks": 5,
      "input_shape": [512, 512],
      "training_data_images": {"raw":X,
                            "gt":Y},
      "validation_data_images": {"raw":X_val,
                              "gt":Y_val},
      "LW_weight_seed": 0,
      "LW_weight_num": 4
  }

  config_rcan.setdefault('epochs', 50)
  config_rcan.setdefault('steps_per_epoch', 256)
  config_rcan.setdefault('initial_learning_rate', 1.288e-4)
  config_rcan.setdefault('loss','SSIML1_loss')
  config_rcan.setdefault('metrics', ['psnr','ssim'])
  config_rcan.setdefault('num_residual_blocks', 3)
  config_rcan.setdefault('num_residual_groups', 5)
  config_rcan.setdefault('channel_reduction', 4)
  config_rcan.setdefault('num_channels', 32)
  config_rcan.setdefault('LW_weight_seed', 0)
  config_rcan.setdefault('LW_weight_num', 4)
  config_rcan.setdefault('batch_size', 2)

  input_shape = config_rcan['input_shape']
  print('Building RCAN model')
  print('  - input_shape =', input_shape)
  for s in ['num_channels',
            'num_residual_blocks',
            'num_residual_groups',
            'channel_reduction']:
      print(f'  - {s} =', config_rcan[s])

  strategy = tf.distribute.MirroredStrategy()
  print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

  with strategy.scope():
      model = build_rcan(
          (*input_shape,1),
          num_channels=config_rcan['num_channels'],
          num_residual_blocks=config_rcan['num_residual_blocks'],
          num_residual_groups=config_rcan['num_residual_groups'],
          channel_reduction=config_rcan['channel_reduction'])

      gpus = get_gpu_count()

      #model = convert_to_multi_gpu_model(model, gpus)
      model.compile(
          optimizer=tf.keras.optimizers.Adam(lr=config_rcan['initial_learning_rate']),
          loss={'mae' : mae,'mse': mse,'SSIM_loss': SSIM_loss,'SSIML1_loss': lambda x, y: SSIML1_loss(x, y, 0.84)}[config_rcan['loss']],
          metrics=[{'psnr': psnr, 'ssim': ssim}[m] for m in config_rcan['metrics']])

      #lambda x, y: raw_loss(x, y, merged_config['alpha'])

  data_gen = DataGenerator(
      input_shape,
      config_rcan['batch_size'],
      transform_function="rotate_and_flip")

  training_data = data_gen.flow(*list(zip([X,Y])))
  validation_data = data_gen.flow(*list(zip([X_val,Y_val])))

  if validation_data is not None:
      checkpoint_filepath = 'weights_{epoch:03d}_{val_loss:.8f}.hdf5'
  else:
      checkpoint_filepath = 'weights_{epoch:03d}_{loss:.8f}.hdf5'

  steps_per_epoch = config_rcan['steps_per_epoch'] //gpus
  validation_steps = None if validation_data is None else steps_per_epoch

  output_dir = model_save_path

  print('Training RCAN model')
  model.fit(
      x=training_data,
      epochs=config_rcan['epochs'],
      #steps_per_epoch=steps_per_epoch,
      validation_data=validation_data,
      #validation_steps=validation_steps,
      verbose=2,
      callbacks=[
          tf.keras.callbacks.ReduceLROnPlateau(verbose=True,factor=0.9,min_delta=0,patience=10,),
          tf.keras.callbacks.EarlyStopping(monitor='ssim', patience=20),
          keras.callbacks.TensorBoard(
              log_dir=str(output_dir),
              write_graph=True),
          ModelCheckpoint(
              str( pathlib.Path(output_dir) / checkpoint_filepath),
              monitor='loss' if validation_data is None else 'val_loss',
              save_best_only=True, verbose = 1, mode='min'),
          TqdmCallback()
      ])


  model.save_weights(model_save_path+'/RCANtrained.h5')

Building RCAN model
  - input_shape = [512, 512]
  - num_channels = 32
  - num_residual_blocks = 5
  - num_residual_groups = 5
  - channel_reduction = 4
Number of devices: 1


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


238

34

Training RCAN model








0epoch [00:00, ?epoch/s]






0batch [00:00, ?batch/s]





  0%|          | 0/50 [00:00<?, ?epoch/s]






  0%|          | 0.00/119 [00:00<?, ?batch/s]

Epoch 1/50


...








  1%|          | 1.00/119 [00:00<01:52, 1.05batch/s, loss=0.27, psnr=18.7, ssim=0.392]






  2%|▏         | 2.00/119 [00:01<01:49, 1.07batch/s, loss=0.302, psnr=17.9, ssim=0.318]






  3%|▎         | 3.00/119 [00:02<01:48, 1.07batch/s, loss=0.303, psnr=17.9, ssim=0.317]






  3%|▎         | 4.00/119 [00:03<01:47, 1.07batch/s, loss=0.308, psnr=17.6, ssim=0.306]






  4%|▍         | 5.00/119 [00:04<01:46, 1.07batch/s, loss=0.309, psnr=17.5, ssim=0.305]






  5%|▌         | 6.00/119 [00:05<01:45, 1.07batch/s, loss=0.311, psnr=17.4, ssim=0.299]






  6%|▌         | 7.00/119 [00:06<01:44, 1.07batch/s, loss=0.305, psnr=17.7, ssim=0.311]






  7%|▋         | 8.00/119 [00:07<01:43, 1.07batch/s, loss=0.304, psnr=17.7, ssim=0.314]






  8%|▊         | 9.00/119 [00:08<01:42, 1.08batch/s, loss=0.304, psnr=17.7, ssim=0.314]






  8%|▊         | 10.0/119 [00:09<01:41, 1.08batch/s, loss=0.306, psnr=17.7, ssim=0.31] 






  9%|▉         | 11.0/119 [00:10<01:39, 1.08batch/s, l


Epoch 00021: val_loss improved from 0.34665 to 0.34628, saving model to /content/drive/MyDrive/Research and Grad/Diamond Program/objective_transfer/deep_learning/NADH_model_0725_cervix_SSIMMAE_kt_hyper_fix/weights_021_0.34628323.hdf5








 42%|████▏     | 21/50 [40:42<56:04, 116.01s/epoch, loss=0.315, psnr=17.2, ssim=0.291, val_loss=0.346, val_psnr=16.6, val_ssim=0.22, lr=0.000129]

119/119 - 117s - loss: 0.3146 - psnr: 17.2307 - ssim: 0.2912 - val_loss: 0.3463 - val_psnr: 16.5837 - val_ssim: 0.2196 - lr: 1.2880e-04 - 117s/epoch - 980ms/step









 42%|████▏     | 21/50 [40:42<56:12, 116.30s/epoch, loss=0.315, psnr=17.2, ssim=0.291, val_loss=0.346, val_psnr=16.6, val_ssim=0.22, lr=0.000129]


## **CARE**

> Indented block



In [ ]:
# TRAINING CARES TEST
from __future__ import print_function, unicode_literals, absolute_import, division
import os
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff
import re
import zipfile
import scipy.io
import random
from tifffile import imread
import argparse
import json
import pickle as pkl
import imageio
import itertools
import jsonschema
import pathlib
from tqdm.utils import IS_WIN
from tqdm.keras import TqdmCallback as _TqdmCallback
import functools
import warnings
import tqdm
import collections
from importlib import import_module
from tensorflow import __version__ as _tf_version
from tensorflow.python.client.device_lib import list_local_devices
from packaging import version
import tensorflow_probability as tfp
import fractions
import numexpr
import tensorflow as tf
from tensorflow.image import ssim as ssim2
from tensorflow.image import ssim_multiscale as msssim
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import constant_op
import keras
import keras.backend as K
from keras.utils.conv_utils import normalize_tuple
from csbdeep.utils import axes_dict, plot_some, plot_history
from csbdeep.internals import nets
from csbdeep.utils.tf import limit_gpu_memory
from csbdeep.models import Config, CARE

def mae(y_true, y_pred):
    '''Mean absolute error'''
    return keras.losses.mae(
        *[keras.backend.batch_flatten(y) for y in [y_true, y_pred]])


def mse(y_true, y_pred):
    '''Mean squared error'''
    return keras.losses.mse(
        *[keras.backend.batch_flatten(y) for y in [y_true, y_pred]])

_tf_version = version.parse(_tf_version)
IS_TF_1 = _tf_version.major == 1
_KERAS = 'keras' if IS_TF_1 else 'tensorflow.keras'

def keras_import(sub=None, *names):
    if sub is None:
        return import_module(_KERAS)
    else:
        mod = import_module('{_KERAS}.{sub}'.format(_KERAS=_KERAS,sub=sub))
        if len(names) == 0:
            return mod
        elif len(names) == 1:
            return getattr(mod, names[0])
        return tuple(getattr(mod, name) for name in names)

def SSIM_loss(y_true, y_pred):
    return 1-((ssim2(y_true, y_pred,1,k2=0.05)+1)*0.5) # change to 2

def SSIML1_loss(y_true, y_pred):
    SSIM = 1-((ssim2(y_true, y_pred,1,k2=0.05)+1)*0.5)
    MAE = keras.losses.mae(
        *[keras.backend.batch_flatten(y) for y in [y_true, y_pred]])
    alpha = 0.84
    return alpha * SSIM + (1-alpha) * MAE # change to 2

def axes_check_and_normalize(axes,length=None,disallowed=None,return_allowed=False):
    """
    S(ample), T(ime), C(hannel), Z, Y, X
    """

    def _raise(e):
        if isinstance(e, BaseException):
            raise e
        else:
            raise ValueError(e)

    allowed = 'STCZYX'
    axes is not None or _raise(ValueError('axis cannot be None.'))
    axes = str(axes).upper()
    consume(a in allowed or _raise(ValueError("invalid axis '%s', must be one of %s."%(a,list(allowed)))) for a in axes)
    disallowed is None or consume(a not in disallowed or _raise(ValueError("disallowed axis '%s'."%a)) for a in axes)
    consume(axes.count(a)==1 or _raise(ValueError("axis '%s' occurs more than once."%a)) for a in axes)
    length is None or len(axes)==length or _raise(ValueError('axes (%s) must be of length %d.' % (axes,length)))
    return (axes,allowed) if return_allowed else axes

def consume(iterator):
    collections.deque(iterator, maxlen=0)

def axes_dict(axes):
    """
    from axes string to dict
    """
    axes, allowed = axes_check_and_normalize(axes,return_allowed=True)
    return { a: None if axes.find(a) == -1 else axes.find(a) for a in allowed }
    # return collections.namedtuple('Axes',list(allowed))(*[None if axes.find(a) == -1 else axes.find(a) for a in allowed ])

def backend_channels_last():
    K = keras_import('backend')
    assert K.image_data_format() in ('channels_first','channels_last')
    return K.image_data_format() == 'channels_last'

def move_channel_for_backend(X,channel):
    if backend_channels_last():
        return np.moveaxis(X, channel, -1)
    else:
        return np.moveaxis(X, channel,  1)

def get_gpu_count():
    '''Returns the number of available GPUs.'''
    return len([x for x in list_local_devices() if x.device_type == 'GPU'])

def psnr(y_true, y_pred):
    '''
    Computs the peak signal-to-noise ratio between two images. Note that the
    maximum signal value is assumed to be 1.
    '''
    p, q = [K.batch_flatten(y) for y in [y_true, y_pred]]
    psnr2 = tf.image.psnr(y_true, y_pred, max_val=1.0)
    return psnr2
    # return -4.342944819 * K.log(K.mean(K.square(p - q), axis=-1))


def ssim(y_true, y_pred):
    '''
    Computes the structural similarity index between two images. Note that the
    maximum signal value is assumed to be 1.
    '''

    return ssim2(y_true, y_pred,1,k2=0.05)

def load_training_data(file, validation_split=0, axes=None, n_images=None, verbose=False):
    """Load training data from file in ``.npz`` format.
    The data file is expected to have the keys:
    - ``X``    : Array of training input images.
    - ``Y``    : Array of corresponding target images.
    - ``axes`` : Axes of the training images.
    Parameters
    ----------
    file : str
        File name
    validation_split : float
        Fraction of images to use as validation set during training.
    axes: str, optional
        Must be provided in case the loaded data does not contain ``axes`` information.
    n_images : int, optional
        Can be used to limit the number of images loaded from data.
    verbose : bool, optional
        Can be used to display information about the loaded images.
    Returns
    -------
    tuple( tuple(:class:`numpy.ndarray`, :class:`numpy.ndarray`), tuple(:class:`numpy.ndarray`, :class:`numpy.ndarray`), str )
        Returns two tuples (`X_train`, `Y_train`), (`X_val`, `Y_val`) of training and validation sets
        and the axes of the input images.
        The tuple of validation data will be ``None`` if ``validation_split = 0``.
    """

    if verbose: print('Loading data...')

    model_save_path = os.path.join(main_path,model_name)
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
    print(is_cuda_gpu_available)
    (X,X_val) = load_data_care('/npz/Cervix_all_data512px.npz')
    (Y,Y_val) = load_data_care('/npz/Cervix_all_data_labels512px.npz')
    axes = 'SXYC'

    if True:
        print(f'Found axes: {axes}')
        print(f'X shape is {tf.shape(X)}; Y shape is {tf.shape(Y)}')

    X = tf.expand_dims(X, axis=-1)
    Y = tf.expand_dims(Y, axis=-1)
    X_t = tf.expand_dims(X_val, axis=-1)
    Y_t = tf.expand_dims(Y_val, axis=-1)
    # assert X.shape == Y.shape
    # assert X.ndim == Y.ndim
    # assert len(axes) == X.ndim
    # assert 'C' in axes
    # if n_images is None:
    #     n_images = X.shape[0]
    # assert X.shape[0] == Y.shape[0]
    # assert 0 < n_images <= X.shape[0]
    # assert 0 <= validation_split < 1

    # X, Y = X[:n_images], Y[:n_images]
    # channel = axes_dict(axes)['C']

    # if validation_split > 0:
    #     #n_val   = int(round(n_images * validation_split))
    #     ''' NEED TO SET VALIDATION SPLIT BETTER '''
    #     n_val = 620
    #     n_train = n_images - n_val
    #     assert 0 < n_val and 0 < n_train
    #     X_t, Y_t = X[-n_val:],  Y[-n_val:]
    #     X,   Y   = X[:n_train], Y[:n_train]
    #     assert X.shape[0] == n_train and X_t.shape[0] == n_val
    #     X_t = move_channel_for_backend(X_t,channel=channel)
    #     Y_t = move_channel_for_backend(Y_t,channel=channel)

    # X = move_channel_for_backend(X,channel=channel)
    # Y = move_channel_for_backend(Y,channel=channel)

    # axes = axes.replace('C','') # remove channel
    # if backend_channels_last():
    #     axes = axes+'C'
    # else:
    #     axes = axes[:1]+'C'+axes[1:]

    data_val = (X_t,Y_t)

    if verbose:
        ax = axes_dict(axes)
        n_train, n_val = len(X), len(X_t) if validation_split>0 else 0
        image_size = tuple( X.shape[ax[a]] for a in axes if a in 'TZYX' )
        n_dim = len(image_size)
        n_channel_in, n_channel_out = X.shape[ax['C']], Y.shape[ax['C']]

        print('number of training images:\t', n_train)
        print('number of validation images:\t', n_val)
        print('image size (%dD):\t\t'%n_dim, image_size)
        print('axes:\t\t\t\t', axes)
        print(f'X shape: {tf.shape(X)}; Y shape: {tf.shape(Y)}')
        print('Loaded data successfully.')
        print('channels in / out:\t\t', n_channel_in, '/', n_channel_out)

    return (X,Y), data_val, axes

def build_CARes(config):
    model = nets.common_unet(
            n_dim           = config.n_dim,
            n_channel_out   = config.n_channel_out,
            prob_out        = config.probabilistic,
            residual        = config.unet_residual,
            n_depth         = config.unet_n_depth,
            kern_size       = config.unet_kern_size,
            n_first         = config.unet_n_first,
            last_activation = config.unet_last_activation,
        )(config.unet_input_shape)

    return model


def compile_model(model, lr):
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lr),
                  loss=SSIML1_loss,
                  metrics = [{'psnr': psnr, 'ssim': ssim}])
    return model



In [ ]:
if True:
  #################################################################################
  ''' Data was Normalized in Local Prep'''

  data_path = MAIN_DIR
  model_name = 'NADH_CAREmodel_0724_cervix_SSIML1_BS50'
  # main_path = '/cluster/tufts/georgakoudi_lab01/nvora01/NV_052622_Denoising/'
  # os.chdir(main_path)

  # if not os.path.exists(os.path.join(main_path,model_name)): os.mkdir(os.path.join(main_path,model_name))
  model_save_path = os.path.join(main_path,model_name)
  print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

  is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
  print(is_cuda_gpu_available)

  (Xc,Yc), (Xc_val,Yc_val), axes = load_training_data(data_path, validation_split=0, axes='SXYC', verbose=True)

  n_channel_in, n_channel_out = 1,1
  config = Config(axes, n_channel_in, n_channel_out,
                  train_steps_per_epoch=100, train_batch_size=2,
                  train_epochs=50, unet_n_depth=2,
                  unet_n_first=32, unet_kern_size=5,
                  train_learning_rate=1e-04,unet_last_activation='relu')
  print(config)

  # CALLBACKS
  # learning rate reducer callback
  reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(verbose=True,factor=0.99,min_delta=0,patience=5,)

  # create a callback that saves the model's weights every said amount of epochs
  epoch_freq = 10
  checkpoint_filepath = 'weights_{epoch:03d}_{val_loss:.8f}.hdf5'
  cp_callback = tf.keras.callbacks.ModelCheckpoint(
      str( pathlib.Path(model_save_path) / checkpoint_filepath),
      monitor='val_loss',
      save_best_only=True, verbose=1, mode='min')
  stop_early = tf.keras.callbacks.EarlyStopping(monitor='ssim', patience=20)
  # BUILD
  model = build_CARes(config)
  model = compile_model(model, config.train_learning_rate)
  model.summary()
  # TRAIN

  history = model.fit(Xc[:9], Yc[:9],
                      batch_size=config.train_batch_size,
                      epochs=config.train_epochs,
                      shuffle=True,
                      validation_data=(Xc_val, Yc_val),
                      callbacks=[reduce_lr, cp_callback,stop_early])

  model.save_weights(model_save_path+'/CAREtrained.h5')

##**Testing**

In [ ]:
def apply(model, data, overlap_shape=None, verbose=False):
    '''
    Applies a model to an input image. The input image stack is split into
    sub-blocks with model's input size, then the model is applied block by
    block. The sizes of input and output images are assumed to be the same
    while they can have different numbers of channels.
    Parameters
    ----------
    model: keras.Model
        Keras model.
    data: array_like or list of array_like
        Input data. Either an image or a list of images.
    overlap_shape: tuple of int or None
        Overlap size between sub-blocks in each dimension. If not specified,
        a default size ((32, 32) for 2D and (2, 32, 32) for 3D) is used.
        Results at overlapped areas are blended together linearly.
    Returns
    -------
    ndarray
        Result image.
    '''

    model_input_image_shape = tuple([None, 256, 256, 1][1:-1])
    # model_input_image_shape = tuple(model.input.shape.as_list()[1:-1])
    print('model.input.shape.as_list()',model.input.shape.as_list())
    print('model_input_image_shape',model_input_image_shape)
    model_output_image_shape = tuple([None, 256, 256, 1][1:-1])
    # model_output_image_shape = tuple(model.output.shape.as_list()[1:-1])
    if len(model_input_image_shape) != len(model_output_image_shape):
        raise NotImplementedError

    image_dim = len(model_input_image_shape)
    num_input_channels = model.input.shape[-1]
    num_output_channels = model.output.shape[-1]

    scale_factor = tuple(
        fractions.Fraction(o, i) for i, o in zip(
            model_input_image_shape, model_output_image_shape))

    def _scale_tuple(t):
        t = [v * f for v, f in zip(t, scale_factor)]

        if not all([v.denominator == 1 for v in t]):
            raise NotImplementedError

        return tuple(v.numerator for v in t)

    def _scale_roi(roi):
        roi = [slice(r.start * f, r.stop * f)
               for r, f in zip(roi, scale_factor)]

        if not all([
                r.start.denominator == 1 and
                r.stop.denominator == 1 for r in roi]):
            raise NotImplementedError

        return tuple(slice(r.start.numerator, r.stop.numerator) for r in roi)

    if overlap_shape is None:
        if image_dim == 2:
            overlap_shape = (32, 32)
        elif image_dim == 3:
            overlap_shape = (2, 32, 32)
        else:
            raise NotImplementedError
    elif len(overlap_shape) != image_dim:
        raise ValueError(f'Overlap shape must be {image_dim}D; '
                         f'Received shape: {overlap_shape}')

    step_shape = tuple(
        m - o for m, o in zip(
            model_input_image_shape, overlap_shape))

    block_weight = np.ones(
        [m - 2 * o for m, o
         in zip(model_output_image_shape, _scale_tuple(overlap_shape))],
        dtype=np.float32)

    block_weight = np.pad(
        block_weight,
        [(o + 1, o + 1) for o in _scale_tuple(overlap_shape)],
        'linear_ramp'
    )[(slice(1, -1),) * image_dim]

    batch_size = model.gpus if is_multi_gpu_model(model) else 1
    batch = np.zeros(
        (batch_size, *model_input_image_shape, num_input_channels),
        dtype=np.float32)

    if isinstance(data, (list, tuple)):
        input_is_list = True
    else:
        data = [data]
        input_is_list = False

    result = []
    for image in data:
        # add the channel dimension if necessary
        print(image.shape)
        if len(image.shape) == image_dim:
            image = image[..., np.newaxis]

        if (len(image.shape) != image_dim + 1
                or image.shape[-1] != num_input_channels):
            raise ValueError(f'Input image must be {image_dim}D with '
                             f'{num_input_channels} channels; '
                             f'Received image shape: {image.shape}')

        input_image_shape = image.shape[:-1]
        output_image_shape = _scale_tuple(input_image_shape)

        applied = np.zeros(
            (*output_image_shape, num_output_channels), dtype=np.float32)
        sum_weight = np.zeros(output_image_shape, dtype=np.float32)

        num_steps = tuple(
            i // s + (i % s != 0)
            for i, s in zip(input_image_shape, step_shape))

        # top-left corner of each block
        blocks = list(itertools.product(
            *[np.arange(n) * s for n, s in zip(num_steps, step_shape)]))

        for chunk_index in tqdm.trange(
                0, len(blocks), batch_size, disable=not verbose,
                dynamic_ncols=True, ascii=tqdm.utils.IS_WIN):
            rois = []
            for batch_index, tl in enumerate(
                    blocks[chunk_index:chunk_index + batch_size]):
                br = [min(t + m, i) for t, m, i
                      in zip(tl, model_input_image_shape, input_image_shape)]
                r1, r2 = zip(
                    *[(slice(s, e), slice(0, e - s)) for s, e in zip(tl, br)])

                m = image[r1]
                if model_input_image_shape != m.shape[-1]:
                    pad_width = [(0, b - s) for b, s
                                 in zip(model_input_image_shape, m.shape[:-1])]
                    pad_width.append((0, 0))
                    m = np.pad(m, pad_width, 'reflect')

                batch[batch_index] = m
                rois.append((r1, r2))

            p = model.predict(batch, batch_size=batch_size)

            for batch_index in range(len(rois)):
                for channel in range(num_output_channels):
                    p[batch_index, ..., channel] *= block_weight

                r1, r2 = [_scale_roi(roi) for roi in rois[batch_index]]

                applied[r1] += p[batch_index][r2]
                sum_weight[r1] += block_weight[r2]

        for channel in range(num_output_channels):
            applied[..., channel] /= sum_weight

        if applied.shape[-1] == 1:
            applied = applied[..., 0]

        result.append(applied)

    return result if input_is_list else result[0]


In [ ]:
def revert_img(img,original_size, patch_shape, slide):
  # reverts original image and removes overlaps by splitting overlap over 2 images
  step = int(patch_shape-slide)
  reconstructed_arr = np.zeros((original_size,original_size))
  for x in range(img.shape[0]):
    for y in range(img.shape[1]):
      start_x = int(slide/2)
      start_y = int(slide/2)
      end_x = 0
      end_y = 0
      if x == 0:
        start_x = 0
        end_x = int(slide/2)
      if y == 0:
        start_y = 0
        end_y = int(slide/2)
      if x == img.shape[0]-1: end_x = int(slide/2)
      if y == img.shape[1]-1: end_y = int(slide/2)
      x_pos, y_pos = x * step + start_x, y * step + start_y
      # print('x_pos, start_x: ',x_pos, start_x)
      # print('x_pos + step + end_x, start_x+step+end_x: ',x_pos + step + end_x, start_x+step+end_x)
      # print('y_pos, start_y: ',y_pos, start_y)
      # print('y_pos + step + end_y,  start_y+step+end_y: ',y_pos + step + end_y,  start_y+step+end_y)
      # print("end===============x,y: ",x,y)
      reconstructed_arr[x_pos : x_pos + step + end_x, y_pos : y_pos + step + end_y] = img[x, y, start_x:start_x+step+end_x, start_y:start_y+step+end_y]
  return reconstructed_arr

import math
def merge_patches(img, original_size, patch_shape, slide):
  #  merging patches, img is a 3D array of stacked patches
  print("merging patches, img shape: ", img.shape)
  row_len = int(math.sqrt(img.shape[0]))
  patches = np.zeros((row_len,row_len,patch_shape,patch_shape))
  print(img.shape)
  print(patches.shape)
  for r in range(row_len):
      patches[r,:,:,:] = img[r*row_len:r*row_len+row_len,:,:]
  # plt.figure(5)
  # plt.imshow(patches[1,1,:,:])
  return revert_img(patches,original_size,patch_shape, slide)


def MSE(img1, img2):
  # comparing one processed and preprocessed image
  squared_diff = (img1 -img2) ** 2
  summed = np.sum(squared_diff)
  num_pix = img1.shape[0] * img1.shape[1] #img1 and 2 should have same shape
  err = summed / num_pix
  return err

def normalize(img):
  # Normalizing images between 0 and 1 and preserving distribution
  img_norm = (img - np.amin(img))/( np.amax(img)- np.amin(img))
  return img_norm

def plot(img, label, hist=False, size=(10,10)):
  plt.figure(figsize = size)
  if hist==True:
    plt.hist(img.flatten(), bins=120)
  else: plt.imshow(img,cmap="gray")
  plt.title(label)

def normalize_between_zero_and_one(m):
    max_val, min_val = m.max(), m.min()
    diff = max_val - min_val
    return (m - min_val) / diff if diff > 0 else np.zeros_like(m)

def normalize_percentile(img, pmin=0.1, pmax=99.9, clip = True):
  eps=1e-20 # avoid zero division
  mi = np.percentile(img,pmin,axis=None,keepdims=True)
  # mi = np.amin(img)
  # print("mi",mi)
  ma = np.percentile(img,pmax,axis=None,keepdims=True)
  if clip == True: return np.clip((img - mi) / ( ma - mi + eps ), 0, 1)
  return (img - mi) / ( ma - mi + eps )

# def ssim(y_true, y_pred):
#     '''
#     Computes the structural similarity index between two images. Note that the
#     maximum signal value is assumed to be 1.
#     '''

#     return ssim2(y_true, y_pred,1,k2=0.05)
# our SSIM loss
def ssim_our(y_true, y_pred):
  return ssim2(y_true, y_pred,1,k2=0.05)

# RCAN ssim
import keras.backend as K
def ssim_rcan(y_true, y_pred):
    '''
    Computes the structural similarity index between two images. Note that the
    maximum signal value is assumed to be 1.
    References
    ----------
    Image Quality Assessment: From Error Visibility to Structural Similarity
    https://doi.org/10.1109/TIP.2003.819861
    '''

    c1 = 0.01 ** 2
    c2 = 0.03 ** 2

    dim = K.ndim(y_pred) - 2
    if dim not in (2, 3):
        raise NotImplementedError(f'{dim}D SSIM is not suported')

    num_channels = K.int_shape(y_pred)[-1]

    kernel = _get_gaussian_kernel(dim, 11, 1.5)
    conv = K.conv2d if dim == 2 else K.conv3d

    def average(x):
        # channel-wise weighted average using the Gaussian kernel
        return tf.concat(
            [conv(y, kernel) for y in tf.split(x, num_channels, axis=-1)],
            axis=-1)

    ux = average(y_true)
    uy = average(y_pred)

    a = ux * uy
    b = K.square(ux) + K.square(uy)
    c = average(y_true * y_pred)
    d = average(K.square(y_true) + K.square(y_pred))

    lum = (2 * a + c1) / (b + c1)
    cs = (2 * (c - a) + c2) / (d - b + c2)

    return K.mean(K.batch_flatten(lum * cs), axis=-1)

In [ ]:
def predictAndPlot(model, start, end, isPlot=True, merge=True, normalize=False, clip=False):
  pred = model.predict(X[start:end])
  print("shape of predicted image: ", np.shape(pred))
  if len(np.shape(X)) > 3:
    if merge:
      result = [merge_patches(X[start:end,:,:,0],512,256,128), merge_patches(pred[start:end,:,:,0],512,256,128), merge_patches(Y[start:end,:,:,0],512,256,128)]
    else: result = [X[start:end,:,:,0], pred[start:end,:,:,0], Y[start:end,:,:,0]]
  else:
    if merge:
      result = [merge_patches(X[start:end,:,:],512,256,128), merge_patches(pred[start:end,:,:,0],512,256,128), merge_patches(Y[start:end,:,:],512,256,128)]
    else: result = [X[start:end,:,:], pred[start:end,:,:,0], Y[start:end,:,:]]
  if normalize: result = [normalize_percentile(m) for m in result]
  if clip: result = [np.clip(255 * m, 0, 255).astype('uint8') for m in result]
  res_img =  np.concatenate((np.concatenate((result[0],result[1]),axis=1),result[2]),axis=1)
  plot(res_img, "input, restored, gt", size=(30,30))
  # plot(result[0],'input 25x')
  # plot(result[1],'restored')
  # plot(result[2],'gt 40x')
  if isPlot:
    plot(result[0],'hist input 25x',hist=True)
    plot(result[1],'hist restored',hist=True)
    plot(result[2],'hist gt 40x',hist=True)
  return result

def evalSSIM(result):
  result1 = [np.expand_dims(m,-1) for m in result]
  raw = tf.convert_to_tensor(np.expand_dims(result1[0],0),dtype=np.float32)
  rest = tf.convert_to_tensor(np.expand_dims(result1[1],0),dtype=np.float32)
  gt = tf.convert_to_tensor(np.expand_dims(result1[2],0),dtype=np.float32)
  # Metric evaluation
  print('raw vs gt=======================')
  print("Our SSIM between raw and ground truth: ",ssim_our(gt,raw).numpy())
  print("RCAN paper SSIM between raw and ground truth: ",ssim_rcan(gt, raw).numpy())
  print('predicted vs gt=======================')
  print("Our SSIM between predicted and ground truth: ",ssim_our(gt, rest).numpy())
  print("RCAN paper SSIM between predicted and ground truth: ",ssim_rcan(gt,rest).numpy())
  print('predicted vs raw=======================')
  print("Our SSIM between predicted and raw: ",ssim_our(raw, rest).numpy())
  print("RCAN paper SSIM between predicted and raw: ",ssim_rcan(raw, rest).numpy())

In [ ]:
# RCAN TEST

model_test_rcan = build_rcan(
          (*input_shape,1),
          num_channels=config_rcan['num_channels'],
          num_residual_blocks=config_rcan['num_residual_blocks'],
          num_residual_groups=config_rcan['num_residual_groups'],
          channel_reduction=config_rcan['channel_reduction'])
model_test_rcan.load_weights(model_save_path+'/RCANtrained.h5')
#input full size images
result = predictAndPlot(model_test_rcan, 0, 9, isPlot=True, merge=True, normalize=False, clip=False)
evalSSIM(result)

# pred = apply(model_test,merged_pred,overlap_shape=(32,32), verbose=True)


In [ ]:
# CARE TEST
model_test_care = build_CARes(config)
model_test_care.load_weights(model_save_path+'/CAREtrained.h5')
result = predictAndPlot(model_test_care, 0, 9, isPlot=True, merge=True, normalize=False, clip=False)
evalSSIM(result)


In [ ]:
test = apply(model_test_care, merge_patches(X[0:9,:,:],512,256,128))

In [ ]:
print(np.shape(test))
plot(test,'hist input 25x',hist=False)
plot(test,'hist input 25x',hist=True)

In [ ]:
result = predictAndPlot(model_test_care, 0, 9, isPlot=True, merge=True, normalize=False, clip=False)